In [1]:
include("src/sets.jl")

using Main.HllSets: HllSet

In [2]:
struct Entity{P}
    hll::HllSet{P}
    grad::Any
    op::Any
    
    # Constructor with keyword arguments
    function Entity{P}(hll::HllSet{P}; grad=nothing, op=nothing) where {P}
        new{P}(hll, grad, op)
    end
end

In [3]:
hll1 = HllSet{10}()
entity1 = Entity{10}(hll1)

Entity{10}(HllSet{10}()
, nothing, nothing)

In [4]:
using Base64

function vector_to_string(vec::Vector{UInt64})::String
    # Convert the vector to a byte array
    byte_array = reinterpret(UInt8, vec)
    # Convert the byte array to a string
    return String(byte_array)
end

function string_to_vector(str::String)::Vector{UInt64}
    # Convert the string back to a byte array
    byte_array = Vector{UInt8}(str)
    # Convert the byte array back to a Vector{UInt64}
    return reinterpret(UInt64, byte_array)
end

string_to_vector (generic function with 1 method)

In [5]:
# Example vector
vec = [1, 2, 3, 4, 5]

# Convert vector to UInt64
vec_uint64 = Vector{UInt64}(vec)

# Convert vector to string
str = vector_to_string(vec_uint64)
println("Encoded string: ", str)

# Convert string back to vector
decoded_vec = string_to_vector(str)
println("Decoded vector: ", decoded_vec)

# Check if the roundtrip conversion is lossless
println("Roundtrip conversion is lossless: ", vec_uint64 == decoded_vec)

# Check if the roundtrip conversion is lossless
println("Roundtrip conversion is lossless: ", vec_uint64 == decoded_vec)

Encoded string:                                    
Decoded vector: UInt64[0x0000000000000001, 0x0000000000000002, 0x0000000000000003, 0x0000000000000004, 0x0000000000000005]
Roundtrip conversion is lossless: true
Roundtrip conversion is lossless: true


In [6]:
using PyCall
# using Redis

redis = pyimport("redis")
query = pyimport("redis.commands.search.query")
r = redis.Redis(host="localhost", port=6379)

PyObject Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [7]:
using PyCall

# Import the Python Redis client
redis = pyimport("redis")

# Connect to Redis
r = redis.Redis(host="localhost", port=6379, db=0)

# Define the dictionary
dict = Dict("myfield" => "some value", "another_field" => "another value")

# Convert the dictionary to a list of key-value pairs
args = Vector{String}()
for (k, v) in dict
    push!(args, k)
    push!(args, v)
end

# Call the fcall method with the list of arguments
r.fcall("my_hset", "1", "myhash", args...)

# Retrieve and print the hash
result = r.hgetall("myhash")
println(result)

Dict{Any, Any}("myfield" => "some value", "_last_modified_" => "1723518544", "another_field" => "another value")


## Testing bit_ops and redis_rand commands

In [8]:
using JSON3

# Define the vectors
vector1 = [11, 21, 63, 114]
vector2 = [411, 223, 200, 123]

# Convert vectors to JSON strings
vector1_json = JSON3.write(vector1)
vector2_json = JSON3.write(vector2)

# Call the Lua script with the vectors and the operation
operation = "XOR"  # Change to "OR" or "XOR" as needed
result_json = r.fcall("bit_ops", 0, vector1_json, vector2_json, operation)

# Convert the result from JSON string to Julia array
result = JSON3.read(result_json)

println(result)

result_json = r.fcall("redis_rand", 0, "masha", 10)
println(JSON3.read(result_json))

[400, 202, 247, 9]
{
     "first_p_bits": 71,
   "trailing_zeros": 0,
    "random_number": 1407721543
}
